In [6]:
import cv2 as cv
import numpy as np
import mediapipe as mp
cap = cv.VideoCapture()
cap = cv.VideoCapture(0, cv.CAP_DSHOW)  # 비디오 캡처 객체 생성

mp_hand = mp.solutions.hands  # Mediapipe Hands 모듈
mp_drawing = mp.solutions.drawing_utils  # Mediapipe 그리기 관련 유틸리티 모듈
mp_styles = mp.solutions.drawing_styles  # Mediapipe 그리기 스타일 모듈

hand = mp_hand.Hands(max_num_hands=2, static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)  # Hands 객체 생성


# 로고 이미지 읽고 크기 조정
logo = cv.imread('C:/chan/piano p/piano.png')
frame_width = int(cap.get(3)) # 프레임 가로 길이
frame_height = int(cap.get(4)) # 프레임 세로 길이
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

# 세로 초록색 선의 좌표 계산
segment_width = frame_width / 14  # 초록색 선 간격 계산

segment_coordinates = [i * segment_width for i in range(1, 15)]  # 초록색 선의 x 좌표 계산

# 초록색 선의 y 좌표 설정
line_y1 = 280  # 첫 번째 y값
line_y2 = 480  # 두 번째 y값

line_y_1 = 350  # 첫 번째 y값
line_y_2 = 480  # 두 번째 y값

while True:
    ret, frame = cap.read()  # 비디오 프레임 읽기

    # 웹캠 화면 가장 아래에 로고 이미지 합성
    roi = frame[-logo_height:, :, :]
    roi[np.where(mask)] = 0
    roi += logo

    res = hand.process(cv.cvtColor(frame, cv.COLOR_BGR2RGB))  # 프레임에서 손 감지

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            for landmark in hand_landmarks.landmark:
                x = int(landmark.x * frame_width)  # 랜드마크의 x 좌표 계산
                y = int(landmark.y * frame_height)  # 랜드마크의 y 좌표 계산

                if line_y_1 <= y <= line_y_2:  # 랜드마크의 y 좌표가 초록색 선의 y 범위 내에 있는 경우
                    for i in range(len(segment_coordinates)):
                        if x >= segment_coordinates[i] and x <= segment_coordinates[i] + segment_width:
                            cv.circle(frame, (x, y), 5, (0, 255, 0), -1)  # 랜드마크 위치에 원 그리기
                            overlay = frame.copy()
                            cv.rectangle(overlay, (int(segment_coordinates[i]), line_y1),
                                          (int(segment_coordinates[i] + segment_width), line_y2),
                                          (0, 255, 255), -1)  # 랜드마크 위치에 불투명한 노란색 사각형 그리기
                            alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                            cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)  # 원본 프레임에 오버레이 적용
                            break

    cv.imshow('WebCam', cv.flip(frame, 1))  # 웹캠 화면 표시

    if cv.waitKey(1) == ord('q'):  # 'q' 키를 누르면 종료
        break

cap.release()  # 비디오 캡처 객체 해제
cv.destroyAllWindows()  # 창 닫기


In [23]:
import cv2 as cv
import mediapipe as mp

# 비디오 캡처 객체 생성
cap = cv.VideoCapture(0)
cap = cv.VideoCapture(0, cv.CAP_DSHOW)

# Mediapipe Hands 모듈
mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# 로고 이미지 읽고 크기 조정
logo = cv.imread('C:/chan/piano p/piano.png')
frame_width = int(cap.get(3)) # 프레임 가로 길이
frame_height = int(cap.get(4)) # 프레임 세로 길이
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

# Hands 객체 생성
hand = mp_hand.Hands(
    max_num_hands=2,
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

while True:
    # 비디오 프레임 읽기
    ret, frame = cap.read()
    
    # 웹캠 화면 가장 아래에 로고 이미지 합성
    roi = frame[-logo_height:, :, :]
    roi[np.where(mask)] = 0
    roi += logo
    
    if not ret:
        break

    # 손 감지
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    res = hand.process(frame_rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            for landmark in hand_landmarks.landmark:
                # 화면 크기에 맞게 랜드마크 좌표 계산
                image_height, image_width, _ = frame.shape
                x = int(landmark.x * image_width)
                y = int(landmark.y * image_height)

                # 초록색 선의 y 좌표 범위 내에 있는 경우에만 처리
                if y >= 280 and y <= 480:
                    # 랜드마크 위치에 원 그리기
                    cv.circle(frame, (x, y), 5, (0, 0, 0), -1)

                    # 초록색 선을 그리기 위한 좌표 계산
                    segment_width = image_width // 14
                    segment_index = int(x // segment_width)
                    segment_x1 = segment_index * segment_width
                    segment_x2 = (segment_index + 1) * segment_width

                    # 초록색 선 그리기
                    cv.rectangle(frame, (segment_x1, 280), (segment_x2, 480), (0, 255, 255), -1)
                    
    # 화면 표시
    cv.imshow('WebCam', cv.flip(frame, 1))  # 웹캠 화면 표시

    # 'q' 키를 누르면 종료
    if cv.waitKey(1) == ord('q'):
        break

# 자원 해제
cap.release()
cv.destroyAllWindows()


In [25]:
import cv2 as cv
import mediapipe as mp
import numpy as np

# 비디오 캡처 객체 생성
cap = cv.VideoCapture(0, cv.CAP_DSHOW)

# Mediapipe Hands 모듈
mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# 로고 이미지 읽고 크기 조정
logo = cv.imread('C:/chan/piano p/piano.png')
frame_width = int(cap.get(3)) # 프레임 가로 길이
frame_height = int(cap.get(4)) # 프레임 세로 길이
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

# Hands 객체 생성
hand = mp_hand.Hands(
    max_num_hands=2,
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

while True:
    # 비디오 프레임 읽기
    ret, frame = cap.read()
    
    # 웹캠 화면 가장 아래에 로고 이미지 합성
    roi = frame[-logo_height:, :, :]
    roi[np.where(mask)] = 0
    roi += logo
    
    if not ret:
        break

    # 손 감지
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    res = hand.process(frame_rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            for landmark in hand_landmarks.landmark:
                # 화면 크기에 맞게 랜드마크 좌표 계산
                image_height, image_width, _ = frame.shape
                x = int(landmark.x * image_width)
                y = int(landmark.y * image_height)

                # 초록색 선의 y 좌표 범위 내에 있는 경우에만 처리
                if y >= 280 and y <= 480:
                    # 랜드마크 위치에 원 그리기
                    cv.circle(frame, (x, y), 5, (0, 255, 0), -1)

                    # 초록색 선을 그리기 위한 좌표 계산
                    segment_width = image_width // 14
                    segment_index = int(x // segment_width)
                    segment_x1 = segment_index * segment_width
                    segment_x2 = (segment_index + 1) * segment_width

                    # 불투명한 노란색 사각형 그리기
                    overlay = frame.copy()
                    cv.rectangle(overlay, (segment_x1, 280), (segment_x2, 480), (0, 255, 255), -1)
                    alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                    cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
                    
    # 화면 표시
    cv.imshow('WebCam', cv.flip(frame, 1))  # 웹캠 화면 표시

    # 'q' 키를 누르면 종료
    if cv.waitKey(1) == ord('q'):
        break

# 자원 해제
cap.release()
cv.destroyAllWindows()


In [26]:
import cv2 as cv
import mediapipe as mp
import numpy as np

# 비디오 캡처 객체 생성
cap = cv.VideoCapture(0, cv.CAP_DSHOW)

# Mediapipe Hands 모듈
mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# 로고 이미지 읽고 크기 조정
logo = cv.imread('C:/chan/piano p/piano.png')
frame_width = int(cap.get(3)) # 프레임 가로 길이
frame_height = int(cap.get(4)) # 프레임 세로 길이
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

# Hands 객체 생성
hand = mp_hand.Hands(
    max_num_hands=2,
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

while True:
    # 비디오 프레임 읽기
    ret, frame = cap.read()
    
    # 웹캠 화면 가장 아래에 로고 이미지 합성
    roi = frame[-logo_height:, :, :]
    roi[np.where(mask)] = 0
    roi += logo
    
    if not ret:
        break

    # 손 감지
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    res = hand.process(frame_rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            for landmark in hand_landmarks.landmark:
                # 화면 크기에 맞게 랜드마크 좌표 계산
                image_height, image_width, _ = frame.shape
                x = int(landmark.x * image_width)
                y = int(landmark.y * image_height)

                # 랜드마크 위치에 원 그리기
                cv.circle(frame, (x, y), 5, (0, 255, 0), -1)

                # 초록색 선을 그리기 위한 좌표 계산
                segment_width = image_width // 14
                segment_index = int(x // segment_width)
                segment_x1 = segment_index * segment_width
                segment_x2 = (segment_index + 1) * segment_width

                # 불투명한 노란색 사각형 그리기
                overlay = frame.copy()
                cv.rectangle(overlay, (segment_x1, 280), (segment_x2, 480), (0, 255, 255), -1)
                alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
                    
    # 화면 표시
    cv.imshow('WebCam', cv.flip(frame, 1))  # 웹캠 화면 표시

    # 'q' 키를 누르면 종료
    if cv.waitKey(1) == ord('q'):
        break

# 자원 해제
cap.release()
cv.destroyAllWindows()


# piano_03

In [2]:
import cv2 as cv
import mediapipe as mp
import numpy as np

# 비디오 캡처 객체 생성
cap = cv.VideoCapture()
cap = cv.VideoCapture(0, cv.CAP_DSHOW)

# Mediapipe Hands 모듈
mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# 로고 이미지 읽고 크기 조정
logo = cv.imread('C:/chan/piano p/piano.png')
frame_width = int(cap.get(3)) # 프레임 가로 길이
frame_height = int(cap.get(4)) # 프레임 세로 길이
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

# Hands 객체 생성
hand = mp_hand.Hands(
    max_num_hands=2,
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

while True:
    # 비디오 프레임 읽기
    ret, frame = cap.read()
    
    # 웹캠 화면 가장 아래에 로고 이미지 합성
    roi = frame[-logo_height:, :, :]
    roi[np.where(mask)] = 0
    roi += logo
    
    if not ret:
        break

    # 손 감지
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    res = hand.process(frame_rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            for landmark in hand_landmarks.landmark:
                # 화면 크기에 맞게 랜드마크 좌표 계산
                image_height, image_width, _ = frame.shape
                x = int(landmark.x * image_width)
                y = int(landmark.y * image_height)

                # 랜드마크 위치에 원 그리기
                cv.circle(frame, (x, y), 5, (0, 255, 0), -1)

                # 초록색 선을 그리기 위한 좌표 계산
                segment_width = image_width // 14
                segment_index = int(x // segment_width)
                segment_x1 = segment_index * segment_width
                segment_x2 = (segment_index + 1) * segment_width

                # 랜드마크 y 좌표가 280에서 480 사이에 있는 경우에만 노란색으로 변경
                if y >= 280 and y <= 480:
                    # 불투명한 노란색 사각형 그리기
                    overlay = frame.copy()
                    cv.rectangle(overlay, (segment_x1, 280), (segment_x2, 480), (0, 255, 255), -1)
                    alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                    cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
                    
    # 화면 표시
    cv.imshow('WebCam', cv.flip(frame, 1))  # 웹캠 화면 표시

    # 'q' 키를 누르면 종료
    if cv.waitKey(1) == ord('q'):
        break

# 자원 해제
cap.release()
cv.destroyAllWindows()


# 랜드마크 8번만 인식

In [1]:
import cv2 as cv
import mediapipe as mp
import numpy as np

# 비디오 캡처 객체 생성
cap = cv.VideoCapture(0, cv.CAP_DSHOW)

# Mediapipe Hands 모듈
mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# 로고 이미지 읽고 크기 조정
logo = cv.imread('C:/chan/piano p/piano.png')
frame_width = int(cap.get(3)) # 프레임 가로 길이
frame_height = int(cap.get(4)) # 프레임 세로 길이
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

# Hands 객체 생성
hand = mp_hand.Hands(
    max_num_hands=2,
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

while True:
    # 비디오 프레임 읽기
    ret, frame = cap.read()
    
    # 웹캠 화면 가장 아래에 로고 이미지 합성
    roi = frame[-logo_height:, :, :]
    roi[np.where(mask)] = 0
    roi += logo
    
    if not ret:
        break

    # 손 감지
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    res = hand.process(frame_rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            landmark = hand_landmarks.landmark[8]  # 8번 랜드마크 좌표 가져오기
            image_height, image_width, _ = frame.shape
            x = int(landmark.x * image_width)
            y = int(landmark.y * image_height)

            # 랜드마크 위치에 원 그리기
            cv.circle(frame, (x, y), 5, (0, 255, 0), -1)

            # 초록색 선을 그리기 위한 좌표 계산
            segment_width = image_width // 14
            segment_index = int(x // segment_width)
            segment_x1 = segment_index * segment_width
            segment_x2 = (segment_index + 1) * segment_width

            # 랜드마크 y 좌표가 280에서 480 사이에 있는 경우에만 노란색으로 변경
            if y >= 280 and y <= 480:
                # 불투명한 노란색 사각형 그리기
                overlay = frame.copy()
                cv.rectangle(overlay, (segment_x1, 280), (segment_x2, 480), (0, 255, 255), -1)
                alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    # 화면 표시
    cv.imshow('WebCam', cv.flip(frame, 1))  # 웹캠 화면 표시

    # 'q' 키를 누르면 종료
    if cv.waitKey(1) == ord('q'):
        break

# 자원 해제
cap.release()
cv.destroyAllWindows()


In [5]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import winsound

# 비디오 캡처 객체 생성
cap = cv.VideoCapture(0, cv.CAP_DSHOW)

# Mediapipe Hands 모듈
mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# 로고 이미지 읽고 크기 조정
logo = cv.imread('C:/chan/piano p/piano.png')
frame_width = int(cap.get(3))  # 프레임 가로 길이
frame_height = int(cap.get(4))  # 프레임 세로 길이
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

# Hands 객체 생성
hand = mp_hand.Hands(
    max_num_hands=2,
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 도레미파솔라시도 음의 주파수 설정
note_frequencies = {
    1: 261.63,  # 도
    2: 293.66,  # 레
    3: 329.63,  # 미
    4: 349.23,  # 파
    5: 392.00,  # 솔
    6: 440.00,  # 라
    7: 493.88  # 시
}

while True:
    # 비디오 프레임 읽기
    ret, frame = cap.read()

    # 웹캠 화면 가장 아래에 로고 이미지 합성
    roi = frame[-logo_height:, :, :]
    roi[np.where(mask)] = 0
    roi += logo

    if not ret:
        break

    # 손 감지
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    res = hand.process(frame_rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            for landmark in hand_landmarks.landmark:
                # 화면 크기에 맞게 랜드마크 좌표 계산
                image_height, image_width, _ = frame.shape
                x = int(landmark.x * image_width)
                y = int(landmark.y * image_height)

                # 랜드마크 위치에 원 그리기
                cv.circle(frame, (x, y), 5, (0, 255, 0), -1)

                # 초록색 선을 그리기 위한 좌표 계산
                segment_width = image_width // 14
                segment_index = int(x // segment_width)
                segment_x1 = segment_index * segment_width
                segment_x2 = (segment_index + 1) * segment_width

                # 랜드마크 y 좌표가 280에서 480 사이에 있는 경우에만 음 출력
                if y >= 280 and y <= 480:
                    note_number = segment_index + 1  # 음 번호 계산
                    if note_number in note_frequencies:
                        frequency = note_frequencies[note_number]
                        duration = 500  # 음 재생 시간 (밀리초)
                        winsound.Beep(int(frequency), duration)

                # 불투명한 노란색 사각형 그리기
                overlay = frame.copy()
                cv.rectangle(overlay, (segment_x1, 280), (segment_x2, 480), (0, 255, 255), -1)
                alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    # 화면 표시
    cv.imshow('WebCam', cv.flip(frame, 1))  # 웹캠 화면 표시

    # 'q' 키를 누르면 종료
    if cv.waitKey(1) == ord('q'):
        break

# 자원 해제
cap.release()
cv.destroyAllWindows()


In [11]:
import cv2 as cv
import mediapipe as mp
import numpy as np

# 비디오 캡처 객체 생성
cap = cv.VideoCapture(0, cv.CAP_DSHOW)

# Mediapipe Hands 모듈
mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# 로고 이미지 읽고 크기 조정
logo = cv.imread('C:/chan/piano p/piano.png')
frame_width = int(cap.get(3)) # 프레임 가로 길이
frame_height = int(cap.get(4)) # 프레임 세로 길이
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

# Hands 객체 생성
hand = mp_hand.Hands(
    max_num_hands=2,
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 도레미파솔라시도 음의 주파수 설정
note_frequencies = {
    1: 261.63,  # 도
    2: 293.66,  # 레
    3: 329.63,  # 미
    4: 349.23,  # 파
    5: 392.00,  # 솔
    6: 440.00,  # 라
    7: 493.88  # 시
}

while True:
    # 비디오 프레임 읽기
    ret, frame = cap.read()
    
    # 웹캠 화면 가장 아래에 로고 이미지 합성
    roi = frame[-logo_height:, :, :]
    roi[np.where(mask)] = 0
    roi += logo
    
    if not ret:
        break

    # 손 감지
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    res = hand.process(frame_rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            landmark = hand_landmarks.landmark[8]  # 8번 랜드마크 좌표 가져오기
            image_height, image_width, _ = frame.shape
            x = int(landmark.x * image_width)
            y = int(landmark.y * image_height)

            # 랜드마크 위치에 원 그리기
            cv.circle(frame, (x, y), 5, (0, 255, 0), -1)

            # 초록색 선을 그리기 위한 좌표 계산
            segment_width = image_width // 14
            segment_index = int(x // segment_width)
            segment_x1 = segment_index * segment_width
            segment_x2 = (segment_index + 1) * segment_width

             # 랜드마크 y 좌표가 280에서 480 사이에 있는 경우에만 음 출력
            if y >= 280 and y <= 480:
                note_number = segment_index + 1  # 음 번호 계산
                if note_number in note_frequencies:
                    frequency = note_frequencies[note_number]
                    duration = 1000  # 음 재생 시간 (밀리초)
                    winsound.Beep(int(frequency), duration)
                # 불투명한 노란색 사각형 그리기
                overlay = frame.copy()
                cv.rectangle(overlay, (segment_x1, 280), (segment_x2, 480), (0, 255, 255), -1)
                alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    # 화면 표시
    cv.imshow('WebCam', cv.flip(frame, 1))  # 웹캠 화면 표시

    # 'q' 키를 누르면 종료
    if cv.waitKey(1) == ord('q'):
        break

# 자원 해제
cap.release()
cv.destroyAllWindows()


In [1]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import winsound

# 비디오 캡처 객체 생성
cap = cv.VideoCapture(0, cv.CAP_DSHOW)

# Mediapipe Hands 모듈
mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# 로고 이미지 읽고 크기 조정
logo = cv.imread('C:/chan/piano p/piano.png')
frame_width = int(cap.get(3)) # 프레임 가로 길이
frame_height = int(cap.get(4)) # 프레임 세로 길이
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

# Hands 객체 생성
hand = mp_hand.Hands(
    max_num_hands=2,
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 도레미파솔라시도 음의 주파수 설정
note_frequencies = {
    1: 261.63,  # 도
    2: 293.66,  # 레
    3: 329.63,  # 미
    4: 349.23,  # 파
    5: 392.00,  # 솔
    6: 440.00,  # 라
    7: 493.88  # 시
}

# 이전에 재생한 음인지 확인하기 위한 변수
previous_note = 0

while True:
    # 비디오 프레임 읽기
    ret, frame = cap.read()
    
    # 웹캠 화면 가장 아래에 로고 이미지 합성
    roi = frame[-logo_height:, :, :]
    roi[np.where(mask)] = 0
    roi += logo
    
    if not ret:
        break

    # 손 감지
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    res = hand.process(frame_rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            landmark = hand_landmarks.landmark[8]  # 8번 랜드마크 좌표 가져오기
            image_height, image_width, _ = frame.shape
            x = int(landmark.x * image_width)
            y = int(landmark.y * image_height)

            # 랜드마크 위치에 원 그리기
            cv.circle(frame, (x, y), 5, (0, 255, 0), -1)

            # 초록색 선을 그리기 위한 좌표 계산
            segment_width = image_width // 14
            segment_index = int(x // segment_width)
            segment_x1 = segment_index * segment_width
            segment_x2 = (segment_index + 1) * segment_width

            # 랜드마크 y 좌표가 280에서 480 사이에 있는 경우에만 음 출력
            if y >= 280 and y <= 480:
                note_number = segment_index + 1  # 음 번호 계산
                if note_number in note_frequencies and note_number != previous_note:
                    frequency = note_frequencies[note_number]
                    duration = 500  # 음 재생 시간 (밀리초)
                    winsound.Beep(int(frequency), duration)
                    previous_note = note_number
                # 불투명한 노란색 사각형 그리기
                overlay = frame.copy()
                cv.rectangle(overlay, (segment_x1, 280), (segment_x2, 480), (0, 255, 255), -1)
                alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    # 화면 표시
    cv.imshow('WebCam', cv.flip(frame, 1))  # 웹캠 화면 표시

    # 'q' 키를 누르면 종료
    if cv.waitKey(1) == ord('q'):
        break

# 자원 해제
cap.release()
cv.destroyAllWindows()


# 소리 반만 나옴

In [9]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import winsound
import threading

# 비디오 캡처 객체 생성
cap = cv.VideoCapture(0, cv.CAP_DSHOW)

# 비디오 프레임 읽기를 위한 락 객체
frame_lock = threading.Lock()

# Mediapipe Hands 모듈
mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# 로고 이미지 읽고 크기 조정
logo = cv.imread('C:/chan/piano p/piano.png')
frame_width = int(cap.get(3)) # 프레임 가로 길이
frame_height = int(cap.get(4)) # 프레임 세로 길이
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

# Hands 객체 생성
hand = mp_hand.Hands(
    max_num_hands=2,
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 도레미파솔라시도 음의 주파수 설정
note_frequencies = {
    1: 261.63,  # 도
    2: 293.66,  # 레
    3: 329.63,  # 미
    4: 349.23,  # 파
    5: 392.00,  # 솔
    6: 440.00,  # 라
    7: 493.88  # 시
}

# 이전에 재생한 음인지 확인하기 위한 변수
previous_note = 0

# 비디오 프레임을 읽고 화면을 표시하는 함수
def read_and_show():
    while True:
        # 비디오 프레임 읽기
        ret, frame = cap.read()

        # 웹캠 화면 가장 아래에 로고 이미지 합성
        roi = frame[-logo_height:, :, :]
        roi[np.where(mask)] = 0
        roi += logo

        if not ret:
            break

        # 화면 표시
        cv.imshow('WebCam', cv.flip(frame, 1))  # 웹캠 화면 표시

        # 'q' 키를 누르면 종료
        if cv.waitKey(1) == ord('q'):
            break

# 비디오 프레임을 읽고 화면을 표시하는 쓰레드 생성
video_thread = threading.Thread(target=read_and_show)
video_thread.daemon = True
video_thread.start()

while True:
    # 비디오 프레임 읽기
    ret, frame = cap.read()

    if not ret:
        break

    # 손 감지
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    res = hand.process(frame_rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            landmark = hand_landmarks.landmark[8]  # 8번 랜드마크 좌표 가져오기
            image_height, image_width, _ = frame.shape
            x = int(landmark.x * image_width)
            y = int(landmark.y * image_height)

            # 랜드마크 위치에 원 그리기
            cv.circle(frame, (x, y), 5, (0, 255, 0), -1)

            # 초록색 선을 그리기 위한 좌표 계산
            segment_width = image_width // 14
            segment_index = int(x // segment_width)
            segment_x1 = segment_index * segment_width
            segment_x2 = (segment_index + 1) * segment_width

            # 랜드마크 y 좌표가 280에서 480 사이에 있는 경우에만 음 출력
            if y >= 280 and y <= 480:
                note_number = segment_index + 1  # 음 번호 계산
                if note_number in note_frequencies and note_number != previous_note:
                    frequency = note_frequencies[note_number]
                    duration = 200  # 음 재생 시간 (밀리초)
                    winsound.Beep(int(frequency), duration)
                    previous_note = note_number
                # 불투명한 노란색 사각형 그리기
                overlay = frame.copy()
                cv.rectangle(overlay, (segment_x1, 280), (segment_x2, 480), (0, 255, 255), -1)
                alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    # 화면 표시
    cv.imshow('WebCam', cv.flip(frame, 1))  # 웹캠 화면 표시

    # 'q' 키를 누르면 종료
    if cv.waitKey(1) == ord('q'):
        break

# 자원 해제
cap.release()
cv.destroyAllWindows()


In [1]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import winsound

# 비디오 캡처 객체 생성
cap = cv.VideoCapture()
cap = cv.VideoCapture(0, cv.CAP_DSHOW)

# Mediapipe Hands 모듈
mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# 로고 이미지 읽고 크기 조정
logo = cv.imread('C:/chan/piano p/piano.png')
frame_width = int(cap.get(3)) # 프레임 가로 길이
frame_height = int(cap.get(4)) # 프레임 세로 길이
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

# Hands 객체 생성
hand = mp_hand.Hands(
    max_num_hands=2,
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 도레미파솔라시도 음의 주파수 설정
note_frequencies = {
    '13': 261.63,
    '12': 293.66,
    '11': 329.63,
    '10': 349.23,
    '9': 392.00,
    '8': 440.00,
    '7': 493.88,
    '6': 261.63*2,
    '5': 293.66*2,
    '4': 329.63*2,
    '3': 349.23*2,
    '2': 392.00*2,
    '1': 440.00*2,
    '0': 493.88*2,
}

# 이전에 재생한 음인지 확인하기 위한 변수
previous_note = 0

# 노란색 선의 y 좌표 설정
line_y1 = 380  # 첫 번째 y값
line_y2 = 480  # 두 번째 y값

while True:
    # 비디오 프레임 읽기
    ret, frame = cap.read()
    
    # 웹캠 화면 가장 아래에 로고 이미지 합성
    roi = frame[-logo_height:, :, :]
    roi[np.where(mask)] = 0
    roi += logo
    
    if not ret:
        break

    # 손 감지
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    res = hand.process(frame_rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            landmark = hand_landmarks.landmark[8]  # 8번 랜드마크 좌표 가져오기
            image_height, image_width, _ = frame.shape
            x = int(landmark.x * image_width)
            y = int(landmark.y * image_height)
            if line_y1 <= y <= line_y2:  # 랜드마크의 y 좌표가 초록색 선의 y 범위 내에 있는 경우
                    for i in range(len(segment_coordinates)):
                        if x >= segment_coordinates[i] and x <= segment_coordinates[i] + segment_width:
                            cv.circle(frame, (x, y), 5, (0, 255, 0), -1)  # 랜드마크 위치에 원 그리기
                            overlay = frame.copy()
                            cv.rectangle(overlay, (int(segment_coordinates[i]), line_y1),
                                          (int(segment_coordinates[i] + segment_width), line_y2),
                                          (0, 255, 255), -1)  # 랜드마크 위치에 불투명한 노란색 사각형 그리기
                            alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                            cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)  # 원본 프레임에 오버레이 적용
                            break
            # 랜드마크 위치에 원 그리기
            # cv.circle(frame, (x, y), 5, (0, 255, 0), -1)

            # 음 재생 구간 나누기
            segment_width = image_width // len(note_frequencies)
            note_number = str(x // segment_width)

            # 랜드마크 y 좌표가 280에서 480 사이에 있는 경우에만 음 출력
            if y >= 280 and y <= 480 and note_number != previous_note:
                frequency = note_frequencies[note_number]
                duration = 300  # 음 재생 시간 (밀리초)
                winsound.Beep(int(frequency), duration)
                previous_note = note_number

            # # 불투명한 노란색 사각형 그리기
            # overlay = frame.copy()
            # cv.rectangle(overlay, ((note_number - 1) * segment_width, 280), (note_number * segment_width, 480), (0, 255, 255), -1)
            # alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
            # cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    # 화면 표시
    cv.imshow('WebCam', cv.flip(frame, 1))  # 웹캠 화면 표시

    # 'q' 키를 누르면 종료
    if cv.waitKey(1) == ord('q'):
        break

# 자원 해제
cap.release()
cv.destroyAllWindows()


NameError: name 'segment_coordinates' is not defined

In [1]:
import cv2 as cv
import numpy as np
import mediapipe as mp
import winsound
import threading

cap = cv.VideoCapture(0)
cap = cv.VideoCapture(0, cv.CAP_DSHOW)

mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

hand = mp_hand.Hands(max_num_hands=2, static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 각 음에 해당하는 주파수
notes = {
    '13': 261.63,
    '12': 293.66,
    '11': 329.63,
    '10': 349.23,
    '9': 392.00,
    '8': 440.00,
    '7': 493.88,
    '6': 261.63*2,
    '5': 293.66*2,
    '4': 329.63*2,
    '3': 349.23*2,
    '2': 392.00*2,
    '1': 440.00*2,
    '0': 493.88*2,
}

# 음을 연주하는 함수
def play_note(note, duration):
    frequency = notes[note]
    winsound.Beep(int(frequency), int(duration * 1000))

# 로고 이미지 읽고 크기 조정
logo = cv.imread('./piano.png')
frame_width = int(cap.get(3)) # 3은 CAP_PROP_FRAME_WIDTH를 나타내며, 프레임의 가로 길이
frame_height = int(cap.get(4)) # # 4는 CAP_PROP_FRAME_HEIGHT를 나타내며, 프레임의 세로 길이
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV) # 로고 이미지의 흑백 이미지에서 임계값(threshold)을 기준으로 이진화된 마스크(mask)를 생성

# 세로 초록색 선의 좌표 계산
segment_width = frame_width / 14

segment_coordinates = [i * segment_width for i in range(1, 14)]

# 노란색 선의 y 좌표 설정
line_y1 = 380  # 첫 번째 y값
line_y2 = 480  # 두 번째 y값

# 비디오 프레임을 읽고 화면을 표시하는 함수
def read_and_show():
    while True:
        # 비디오 프레임 읽기
        ret, frame = cap.read()

        # 웹캠 화면 가장 아래에 로고 이미지 합성
        roi = frame[-logo_height:, :, :]
        roi[np.where(mask)] = 0
        roi += logo

        if not ret:
            break

        # 화면 표시
        cv.imshow('WebCam', cv.flip(frame, 1))  # 웹캠 화면 표시

        # 'q' 키를 누르면 종료
        if cv.waitKey(1) == ord('q'):
            break

# 비디오 프레임을 읽고 화면을 표시하는 쓰레드 생성
video_thread = threading.Thread(target=read_and_show)
video_thread.daemon = True
video_thread.start()

while True:
    ret, frame = cap.read()

    res = hand.process(cv.cvtColor(frame, cv.COLOR_BGR2RGB))

    if res.multi_hand_landmarks: # 프레임에서 손 랜드마크가 감지되었는지를 확인
        for landmarks in res.multi_hand_landmarks:
            x = int(landmarks.landmark[8].x * frame_width)
            y = int(landmarks.landmark[8].y * frame_height)
            print(y)
            if line_y1 <= y <= line_y2:
                for i in range(len(segment_coordinates)):
                    if x >= segment_coordinates[i] and x <= segment_coordinates[i] + segment_width:
                        overlay = frame.copy()
                        cv.rectangle(overlay, (int(segment_coordinates[i]), line_y1),
                                        (int(segment_coordinates[i] + segment_width), line_y2),
                                        (0, 255, 255), -1)
                        alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                        play_note(f'{i}', 0.5)
                        cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
                        break

    cv.imshow('WebCam', cv.flip(frame, 1))

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

546
515
544
507
503
487
472
481
494
497
499
496
484
471
418
407
484
474
489
501
507
503
509
501
508
498
498
506
490
433
453
425
411
453
194
475
424
418
400
436
444
458
444
445
435
441
443
448
453
454
452
451
480
426
393
420
430
431
441
433
433
436
437
432
446
450
442
437
447
454
455
455
454
460
456
455
453
456
457
457
456
465
463
452
344
400
483
493
487
430
334


In [1]:
import cv2 as cv
import numpy as np
import mediapipe as mp
import winsound
import threading

cap = cv.VideoCapture(0)
cap = cv.VideoCapture(0, cv.CAP_DSHOW)

mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

hand = mp_hand.Hands(max_num_hands=2, static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 각 음에 해당하는 주파수
notes = {
    '13': 261.63,
    '12': 293.66,
    '11': 329.63,
    '10': 349.23,
    '9': 392.00,
    '8': 440.00,
    '7': 493.88,
    '6': 261.63*2,
    '5': 293.66*2,
    '4': 329.63*2,
    '3': 349.23*2,
    '2': 392.00*2,
    '1': 440.00*2,
    '0': 493.88*2,
}

# 음을 연주하는 함수
def play_note_thread(note, duration):
    frequency = notes[note]
    winsound.Beep(int(frequency), int(duration * 1000))

# 로고 이미지 읽고 크기 조정
logo = cv.imread('./piano.png')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV) # 로고 이미지의 흑백 이미지에서 임계값(threshold)을 기준으로 이진화된 마스크(mask)를 생성

# 세로 초록색 선의 좌표 계산
segment_width = frame_width / 14

segment_coordinates = [i * segment_width for i in range(14)]

# 노란색 선의 y 좌표 설정
line_y1 = 380  # 첫 번째 y값
line_y2 = 480  # 두 번째 y값

# 비디오 프레임을 읽고 화면을 표시하는 함수
def read_and_show():
    while True:
        # 비디오 프레임 읽기
        ret, frame = cap.read()

        # 웹캠 화면 가장 아래에 로고 이미지 합성
        roi = frame[-logo_height:, :, :]
        roi[np.where(mask)] = 0
        roi += logo

        if not ret:
            break

        # 화면 표시
        # cv.imshow('WebCam', cv.flip(frame, 1))  # 웹캠 화면 표시

        # 'q' 키를 누르면 종료
        if cv.waitKey(1) == ord('q'):
            break

# 비디오 프레임을 읽고 화면을 표시하는 쓰레드 생성
video_thread = threading.Thread(target=read_and_show)
video_thread.daemon = True
video_thread.start()

while True:
    ret, frame = cap.read()

    res = hand.process(cv.cvtColor(frame, cv.COLOR_BGR2RGB))

    if res.multi_hand_landmarks: # 프레임에서 손 랜드마크가 감지되었는지를 확인
        for landmarks in res.multi_hand_landmarks:
            x = int(landmarks.landmark[8].x * frame_width)
            y = int(landmarks.landmark[8].y * frame_height)

            if line_y1 <= y <= line_y2:
                for i in range(len(segment_coordinates)):
                    if segment_coordinates[i] <= x < segment_coordinates[i+1]:
                        overlay = frame.copy()
                        cv.rectangle(overlay, (int(segment_coordinates[i]), line_y1),
                                        (int(segment_coordinates[i+1]), line_y2),
                                        (0, 255, 255), -1)
                        alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                        cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
                        threading.Thread(target=play_note_thread, args=(str(i), 0.3)).start()
                        break

    cv.imshow('WebCam', cv.flip(frame, 1))

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


Exception in thread Thread-5:
Traceback (most recent call last):
  File "c:\Users\User\anaconda3\envs\hee\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\User\anaconda3\envs\hee\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_23668\1706978533.py", line 68, in read_and_show
TypeError: 'NoneType' object is not subscriptable


In [1]:
import cv2 as cv
import numpy as np
import mediapipe as mp
import winsound
import threading

cap = cv.VideoCapture(0, cv.CAP_DSHOW)

mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

hand = mp_hand.Hands(max_num_hands=2, static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 각 음에 해당하는 주파수
notes = {
    '13': 261.63,
    '12': 293.66,
    '11': 329.63,
    '10': 349.23,
    '9': 392.00,
    '8': 440.00,
    '7': 493.88,
    '6': 261.63*2,
    '5': 293.66*2,
    '4': 329.63*2,
    '3': 349.23*2,
    '2': 392.00*2,
    '1': 440.00*2,
    '0': 493.88*2,
}

# 음을 연주하는 함수
def play_note_thread(note, duration):
    frequency = notes[note]
    winsound.Beep(int(frequency), int(duration * 1000))

# 로고 이미지 읽고 크기 조정
logo = cv.imread('./piano.png')
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV) # 로고 이미지의 흑백 이미지에서 임계값(threshold)을 기준으로 이진화된 마스크(mask)를 생성

# 세로 초록색 선의 좌표 계산
segment_width = frame_width / 14

segment_coordinates = [i * segment_width for i in range(14)]

# 노란색 선의 y 좌표 설정
line_y1 = 380  # 첫 번째 y값
line_y2 = 480  # 두 번째 y값

# 비디오 프레임을 읽고 화면을 표시하는 함수
def read_and_show():
    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # 웹캠 화면 가장 아래에 로고 이미지 합성
        roi = frame[-logo_height:, :, :]
        roi[np.where(mask)] = 0
        roi += logo

        # 화면 표시
        cv.imshow('WebCam', cv.flip(frame, 1))  # 웹캠 화면 표시

        # 'q' 키를 누르면 종료
        if cv.waitKey(1) == ord('q'):
            break

# 비디오 프레임을 읽고 화면을 표시하는 쓰레드 생성
video_thread = threading.Thread(target=read_and_show)
video_thread.daemon = True
video_thread.start()

while True:
    ret, frame = cap.read()

    res = hand.process(cv.cvtColor(frame, cv.COLOR_BGR2RGB))

    if res.multi_hand_landmarks: # 프레임에서 손 랜드마크가 감지되었는지를 확인
        for landmarks in res.multi_hand_landmarks:
            x = int(landmarks.landmark[8].x * frame_width)
            y = int(landmarks.landmark[8].y * frame_height)

            if line_y1 <= y <= line_y2:
                for i in range(len(segment_coordinates)):
                    if segment_coordinates[i] <= x <= segment_coordinates[i]+ segment_width:
                        overlay = frame.copy()
                        cv.rectangle(overlay, (int(segment_coordinates[i]), line_y1),
                                        (int(segment_coordinates[i+1]), line_y2),
                                        (0, 255, 255), -1)
                        alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                        cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
                        threading.Thread(target=play_note_thread, args=(str(i), 0.3)).start()
                        break

    cv.imshow('WebCam', cv.flip(frame, 1))

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


IndexError: list index out of range

In [7]:
import cv2 as cv
import numpy as np
import mediapipe as mp
import winsound
import threading

cap = cv.VideoCapture(0, cv.CAP_DSHOW)

mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

hand = mp_hand.Hands(max_num_hands=2, static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 각 음에 해당하는 주파수
notes = {
    '13': 261.63,
    '12': 293.66,
    '11': 329.63,
    '10': 349.23,
    '9': 392.00,
    '8': 440.00,
    '7': 493.88,
    '6': 261.63*2,
    '5': 293.66*2,
    '4': 329.63*2,
    '3': 349.23*2,
    '2': 392.00*2,
    '1': 440.00*2,
    '0': 493.88*2,
}

# 음을 연주하는 함수
def play_note_thread(note, duration):
    frequency = notes[note]
    winsound.Beep(int(frequency), int(duration * 1000))

# 로고 이미지 읽고 크기 조정
logo = cv.imread("C:/chan/piano p/piano.png")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV) # 로고 이미지의 흑백 이미지에서 임계값(threshold)을 기준으로 이진화된 마스크(mask)를 생성

# 세로 초록색 선의 좌표 계산
segment_width = frame_width / 14
segment_coordinates = [i * segment_width for i in range(14)]

# 노란색 선의 y 좌표 설정
line_y1 = 380  # 첫 번째 y값
line_y2 = 480  # 두 번째 y값

while True:
    ret, frame = cap.read()

    if not ret:
        break

    res = hand.process(cv.cvtColor(frame, cv.COLOR_BGR2RGB))

    if res.multi_hand_landmarks: # 프레임에서 손 랜드마크가 감지되었는지를 확인
        for landmarks in res.multi_hand_landmarks:
            x = int(landmarks.landmark[8].x * frame_width)
            y = int(landmarks.landmark[8].y * frame_height)

            if line_y1 <= y <= line_y2:
                for i in range(len(segment_coordinates) - 1):
                    if segment_coordinates[i] <= x < segment_coordinates[i+1]:
                        overlay = frame.copy()
                        cv.rectangle(overlay, (int(segment_coordinates[i]), line_y1),
                                        (int(segment_coordinates[i+1]), line_y2),
                                        (0, 255, 255), -1)
                        alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                        cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
                        threading.Thread(target=play_note_thread, args=(str(i), 0.3)).start()
                        break

    cv.imshow('WebCam', cv.flip(frame, 1))

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


In [1]:
import cv2 as cv
import numpy as np
import mediapipe as mp
import winsound
import threading
cap = cv.VideoCapture()
cap = cv.VideoCapture(0, cv.CAP_DSHOW)

mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hand = mp_hand.Hands(max_num_hands=2, static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 각 음에 해당하는 주파수
notes = {
    '13': 261.63,
    '12': 293.66,
    '11': 329.63,
    '10': 349.23,
    '9': 392.00,
    '8': 440.00,
    '7': 493.88,
    '6': 261.63*2,
    '5': 293.66*2,
    '4': 329.63*2,
    '3': 349.23*2,
    '2': 392.00*2,
    '1': 440.00*2,
    '0': 493.88*2,
}

# 음을 연주하는 함수
def play_note_thread(note, duration):
    frequency = notes[note]
    winsound.Beep(int(frequency), int(duration * 1000))

# 로고 이미지 읽기
logo = cv.imread('piano.png', cv.IMREAD_UNCHANGED)
logo_height, logo_width = logo.shape[:2]

while True:
    ret, frame = cap.read()

    if not ret:
        break

    res = hand.process(cv.cvtColor(frame, cv.COLOR_BGR2RGB))

    if res.multi_hand_landmarks: # 프레임에서 손 랜드마크가 감지되었는지를 확인
        for landmarks in res.multi_hand_landmarks:
            x = int(landmarks.landmark[8].x * frame.shape[1])
            y = int(landmarks.landmark[8].y * frame.shape[0])

            if 280 <= y <= 480:
                segment_width = frame.shape[1] // 14
                segment_index = int(x // segment_width)
                segment_x1 = segment_index * segment_width
                segment_x2 = (segment_index + 1) * segment_width

                overlay = frame.copy()
                cv.rectangle(overlay, (segment_x1, 280), (segment_x2, 480), (0, 255, 255), -1)
                alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

                threading.Thread(target=play_note_thread, args=(str(segment_index), 0.3)).start()

    # 로고 이미지 합성
    frame[-logo_height:, :logo_width] = cv.addWeighted(frame[-logo_height:, :logo_width], 1.0, logo[:, :, :3], 0.7, 0)

    cv.imshow('WebCam', cv.flip(frame, 1))

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()



error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:650: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'cv::arithm_op'
